In [1]:
!nvidia-smi

Sat Dec  5 07:48:28 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.45.01    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   51C    P8    11W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 79
model name	: Intel(R) Xeon(R) CPU @ 2.20GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2200.000
cache size	: 56320 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm 3dnowprefetch invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 hle avx2 smep bmi2 erms invpcid rtm rdseed adx smap xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs taa
bogomips	: 4400.00
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 b

In [3]:
!pip install tensorflow-gpu

     |████████████████████████████████| 320.4MB 53kB/s 


In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D,MaxPool2D,Dropout
from tensorflow.keras.layers import Flatten

In [5]:
from tensorflow.python.client import device_lib 
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 2339052353389247912
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 14325500526082121316
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 7375248288400053281
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14613293312
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8498793472155812795
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [6]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_gen=ImageDataGenerator(rescale=1./255,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True)
test_gen=ImageDataGenerator(rescale=1./255)

train_generator = train_gen.flow_from_directory(
        '/content/drive/My Drive/dataset_catsdogs/training_set',
        target_size=(224,224),
        batch_size=32,
        class_mode='binary')
validation_generator = test_gen.flow_from_directory(
        '/content/drive/My Drive/dataset_catsdogs/test_set',
        target_size=(224, 224),
        batch_size=32,
        class_mode='binary')

Found 8009 images belonging to 2 classes.
Found 2010 images belonging to 2 classes.


In [7]:
train_generator.image_shape

(224, 224, 3)

In [8]:
model=Sequential()
model.add(Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=train_generator.image_shape))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.3))

model.add(Flatten())
model.add(Dense(64,activation="relu"))
model.add(Dropout(0.3))
model.add(Dense(1,activation="sigmoid"))

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 16)      448       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 111, 111, 16)      0         
_________________________________________________________________
dropout (Dropout)            (None, 111, 111, 16)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 109, 109, 32)      4640      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 52, 52, 64)        1

In [10]:
model.compile(optimizer="adam",loss="binary_crossentropy",metrics=['accuracy'])

In [12]:
model.fit(
        train_generator,
        steps_per_epoch=len(train_generator),
        epochs=25,
        validation_data=validation_generator,
        validation_steps=len(validation_generator))

Epoch 1/25
251/251 [==============================] - 4260s 17s/step - loss: 0.7066 - accuracy: 0.5250 - val_loss: 0.6723 - val_accuracy: 0.6149
Epoch 2/25
251/251 [==============================] - 106s 421ms/step - loss: 0.6535 - accuracy: 0.6021 - val_loss: 0.6080 - val_accuracy: 0.6662
Epoch 3/25
251/251 [==============================] - 105s 420ms/step - loss: 0.5929 - accuracy: 0.6835 - val_loss: 0.5335 - val_accuracy: 0.7279
Epoch 4/25
251/251 [==============================] - 105s 418ms/step - loss: 0.5654 - accuracy: 0.7031 - val_loss: 0.5176 - val_accuracy: 0.7398
Epoch 5/25
251/251 [==============================] - 105s 418ms/step - loss: 0.5237 - accuracy: 0.7414 - val_loss: 0.5087 - val_accuracy: 0.7517
Epoch 6/25
251/251 [==============================] - 106s 420ms/step - loss: 0.5018 - accuracy: 0.7559 - val_loss: 0.4773 - val_accuracy: 0.7806
Epoch 7/25
251/251 [==============================] - 105s 418ms/step - loss: 0.4744 - accuracy: 0.7800 - val_loss: 0.4664 - 

In [4]:
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import numpy as np
img=load_img('cat.jpg',target_size=(224, 224))
img=img_to_array(img)
img=np.expand_dims(img,axis=0)
result=model.predict(img)
print(result)
if result[0][0]<=0.5:
  print("its a cat.")
else:
  print("its a dog.")

[[0.]]
its a cat.


In [3]:
from tensorflow.keras.models import load_model

model=load_model("cats_dogs.h5")